# image classification with your own data

put images in the class directories for train, valid and test.

for example, with 'HDH' and 'OH' classes 
* ./drive/My Drive/public/train/HDH/*.jpg
* ./drive/My Drive/public/train/OH/*.jpg
* ./drive/My Drive/public/valid/HDH/*.jpg
* ./drive/My Drive/public/valid/OH/*.jpg
* ./drive/My Drive/public/test/HDH/*.jpg
* ./drive/My Drive/public/test/OH/*.jpg

your model will be saved as 'teamX.model'

have fun! 
(by hchoi@handong.edu, Nov. 30, 2019) 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
import os
import warnings
warnings.filterwarnings("ignore")

# try different network architectures
* different kernel size and numbers

In [0]:
resize=(120, 120)
input_size=(100, 100)

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(resize),
        transforms.RandomCrop(input_size), # data augmentation
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.3, 0.3, 0.3])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(resize),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.3, 0.3, 0.3])
    ]),
}
test_transform = data_transforms['valid']

In [0]:
class MyCNN(nn.Module):
    def __init__(self, output_dim=10):
        super(MyCNN,self).__init__()

        self.output_dim=output_dim

        self.cnn_layers = nn.Sequential(
            nn.Conv2d(3,32,3,padding=1), # try with different kernels
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32,32,3,padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(4,4), # 32 x (25x25)
            
            nn.Conv2d(32,16,3,padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Conv2d(16,16,3,padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(5,5) # 16 x (5x5) 
        )
        conv_size = self.get_conv_size(3, input_size)
        self.fc_layer = nn.Sequential(
            nn.Linear(conv_size,100),
            nn.BatchNorm1d(100),
            nn.ReLU(),
            nn.Linear(100,output_dim)
        )       

    def get_conv_size(self, channel, shape):
        o = self.cnn_layers(torch.zeros(1, channel, *shape))
        return int(np.prod(o.size()))
        
    def forward(self,x):
        batch_size, c, h, w = x.data.size()
        out = self.cnn_layers(x)
        out = out.view(batch_size, -1)
        out = self.fc_layer(out)
        return out



# try different learning rates

In [0]:
learning_rate = 0.0005
output_dim=5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = MyCNN(output_dim=output_dim).to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

param_list = list(model.children())
print(param_list)

**data loader**

In [8]:
batch_size = 64 # try different batch_size values
data_dir = './drive/My Drive/public/'
train_dir = 'train'
valid_dir = 'valid1'

train_set = datasets.ImageFolder(data_dir+train_dir, data_transforms['train'])
valid_set = datasets.ImageFolder(data_dir+valid_dir, data_transforms['valid'])

train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                              shuffle=True, num_workers=4)          
valid_loader = torch.utils.data.DataLoader(valid_set, batch_size=batch_size,
                                              shuffle=True, num_workers=4)

train_size = len(train_set)
valid_size = len(valid_set)

class_names = train_set.classes

print(class_names) 
print(f'Train image size: {train_size}')
print(f'Validation image size: {valid_size}')

['ANH', 'HDH', 'Hyoam', 'NTH', 'OH']
Train image size: 4186
Validation image size: 241


**training**
* implement 'early stopping' 

In [9]:
result_dir = 'drive/My Drive/public/results/'
num_epoch = 3 # try with different epochs and find the best epoch

if not os.path.exists(result_dir):
    os.mkdir(result_dir)    
    
for i in range(num_epoch):
    model.train()
    for j, [image,label] in enumerate(train_loader):
        x = image.to(device)
        y_= label.to(device)
        
        optimizer.zero_grad()
        output = model(x)
        loss = loss_func(output,y_)
        loss.backward()
        optimizer.step()
        
        #if j % 30 == 0:
        #   print(i,j, loss.data.cpu())
    
    model.eval()
    hits = 0
    for k,[image,label] in enumerate(valid_loader):
        x = image.to(device)
        y_= label.to(device)

        output = model(x)
        y_est = output.argmax(1)
        hits = hits + sum(y_est == y_).cpu()
    print('Epochs', i, 'Hits', int(hits), 'Accuracy', float(hits/(valid_size+0.0)))    

torch.save(model, result_dir + 'teamX.model')
print('training is done by max_epochs', num_epoch)

Epochs 0 Hits 214 Accuracy 0.8879668116569519
Epochs 1 Hits 217 Accuracy 0.9004149436950684
Epochs 2 Hits 222 Accuracy 0.9211618304252625
training is done by max_epochs 3



---
training is done! 

---

**you can test your model**

In [11]:
test_batch_size = 10
result_dir = 'drive/My Drive/public/results/'
model_name = 'teamX.model'

model = torch.load(result_dir + model_name)
model.to(device)
model.eval()

test_dir = './drive/My Drive/public/valid2'
test_set = datasets.ImageFolder(test_dir, test_transform)
              
test_loader = torch.utils.data.DataLoader(test_set, batch_size=test_batch_size,
                                          shuffle=False, num_workers=4)

hits = 0
for k,[image,label] in enumerate(test_loader):
    x = image.to(device)
    y_= label.to(device)
  
    output = model(x)
    y_est = output.argmax(1)
    print('Target', label.numpy(), 'Prediction', y_est.cpu().numpy())
    hits = hits + sum(y_est == y_)
print('hits', int(hits),'accuracy', float(hits/(len(test_set)+0.0)))

Target [0 0 0 0 0 0 0 0 0 0] Prediction [0 0 0 0 0 0 0 0 0 0]
Target [0 0 0 0 0 0 0 0 0 0] Prediction [0 0 0 0 0 0 0 0 0 0]
Target [0 0 0 0 0 0 0 0 0 0] Prediction [0 0 0 0 0 0 0 0 0 0]
Target [0 0 0 0 0 0 0 0 0 0] Prediction [0 0 0 0 0 0 0 0 0 0]
Target [1 1 1 1 1 1 1 1 1 1] Prediction [1 1 1 1 1 1 1 1 1 1]
Target [1 1 1 1 1 1 1 1 1 1] Prediction [1 1 1 1 1 1 1 1 1 1]
Target [1 1 1 1 1 1 1 1 1 1] Prediction [1 1 1 1 1 2 2 2 2 2]
Target [1 1 1 1 1 1 1 1 1 1] Prediction [2 2 2 2 4 0 0 0 0 0]
Target [2 2 2 2 2 2 2 2 2 2] Prediction [2 2 2 2 2 2 2 2 2 2]
Target [2 2 2 2 2 2 2 2 2 2] Prediction [2 2 2 2 2 2 2 2 2 2]
Target [2 2 2 2 2 2 2 2 2 2] Prediction [2 2 2 2 2 2 2 2 2 2]
Target [2 2 2 2 2 2 2 2 2 2] Prediction [2 2 2 3 4 3 3 3 2 3]
Target [3 3 3 3 3 3 3 3 3 3] Prediction [3 3 3 3 3 3 4 3 4 3]
Target [3 3 3 3 3 3 3 3 3 3] Prediction [3 3 3 3 3 3 3 3 3 3]
Target [3 3 3 3 3 3 3 3 3 3] Prediction [3 3 3 3 3 3 3 3 3 3]
Target [3 3 3 3 3 3 3 3 3 3] Prediction [3 3 4 4 3 0 3 3 2 3]
Target [

# classify one image

In [12]:
from skimage import io

img_name = './drive/My Drive/public/data/test/test1.jpg'
test_img = io.imread(img_name)
test_img = transforms.ToPILImage()(test_img)
test_img = test_transform(test_img)
test_data = test_img.unsqueeze(0).to(device)

output=model(test_data)

class_id = output.argmax(dim=1).cpu().numpy()[0]
print(img_name.split('/')[-1], '==>', class_id, class_names[class_id])

test1.jpg ==> 0 ANH


# classify all images in a directory

In [13]:
from skimage import io
import glob

img_dir = 'drive/My Drive/public/data/test/'
file_list = glob.glob(img_dir + '*.*')
for img_name in file_list:
  test_img = io.imread(img_name)
  test_img = transforms.ToPILImage()(test_img)
  test_img = test_transform(test_img)
  test_data = test_img.unsqueeze(0).to(device)

  output=model(test_data)

  class_id = output.argmax(dim=1).cpu().numpy()[0]
  print(img_name.split('/')[-1], '==>', class_id, class_names[class_id])

D07_ANH22.jpg ==> 0 ANH
D15_OH43.jpg ==> 3 NTH
D06_HDH08.jpg ==> 1 HDH
D08_Hyoam09.jpg ==> 2 Hyoam
D12_NTH15.jpg ==> 3 NTH
D04_OH18.jpeg ==> 4 OH
D12_AHN36.jpg ==> 0 ANH
D05_NTH31.jpg ==> 3 NTH
D17_NTH08.jpg ==> 3 NTH
D12_OH21.jpg ==> 4 OH
D07_HDH45.jpg ==> 1 HDH
D07_HDH30.jpg ==> 1 HDH
D11_NTH15.jpeg ==> 3 NTH
D01_Hyoam41.jpg ==> 2 Hyoam
D06_ANH32.jpg ==> 0 ANH
D09_NTH30.jpg ==> 3 NTH
D13_ANH45.jpg ==> 0 ANH
D13_ANH47.jpg ==> 0 ANH
test1.jpg ==> 0 ANH
D18_Hyoam40.JPG ==> 2 Hyoam
D06_OH29.jpg ==> 4 OH
D11_NTH01.jpeg ==> 3 NTH
D05_NTH45.jpg ==> 3 NTH
D15_OH31.jpg ==> 4 OH
D15_HDH04.JPG ==> 1 HDH
D08_ANH23.jpg ==> 0 ANH
D18_NTH30.jpg ==> 3 NTH
D09_Hyoam31.JPG ==> 2 Hyoam
D03_OH13.jpg ==> 4 OH
D07_HDH17.jpg ==> 0 ANH
D09_Hyoam09.JPG ==> 2 Hyoam
D10_ANH46.jpg ==> 0 ANH
D13_NTH30.jpg ==> 3 NTH
D14_OH12.jpg ==> 4 OH
D07_HDH07.jpg ==> 0 ANH
D09_HDH31.JPG ==> 1 HDH
D08_OH29.jpg ==> 4 OH
D18_ANH48.jpg ==> 0 ANH
D01_OH22.jpg ==> 4 OH
D05_NTH03.jpg ==> 3 NTH
D07_HDH49.jpg ==> 1 HDH
D02_HDH50.jpg 

The end! (hchoi@handong.edu)

---

